In [9]:
%pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
# Carregar os landmarks e rótulos do arquivo .parquet
landmarks_df = pd.read_parquet('/home/viniciuss/Documents/projetos/FER-2013_emotion_detection/landmarks_balanced.parquet')

# Exibir as primeiras linhas do DataFrame para verificar
print("Primeiras linhas do DataFrame:")
print(landmarks_df.head())

Primeiras linhas do DataFrame:
          0         1         2         3         4         5         6  \
0  0.542199  0.738833 -0.070273  0.535373  0.635229 -0.179770  0.532786   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.429018  0.756555 -0.045187  0.431486  0.689657 -0.183432  0.437436   
3  0.455101  0.528905 -0.093874  0.454369  0.438817 -0.144501  0.452669   
4  0.294850  0.448779 -0.197011  0.312964  0.295665 -0.208506  0.317771   

          7         8         9  ...      1395      1396      1397      1398  \
0  0.657359 -0.085466  0.492651  ...  0.566294  0.417931 -0.050389  0.741274   
1  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
2  0.700313 -0.078584  0.430007  ...  0.549186  0.452173 -0.090314  0.760878   
3  0.470683 -0.087341  0.422387  ...  0.478693  0.311043  0.005461  0.612753   
4  0.370636 -0.152194  0.326018  ...  0.462963  0.223220  0.047739  0.674564   

       1399      1400      1401      

In [3]:
# Separar features e targets
X = landmarks_df.drop('label', axis=1).values
y = landmarks_df['label'].values

# Dividir dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizar as features se necessário (dependendo do modelo utilizado)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificar tamanhos dos conjuntos
print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")

Tamanho do conjunto de treino: 12265 amostras
Tamanho do conjunto de teste: 3067 amostras


# Treinamentos SVC, LDA e RFC

In [13]:
# Inicializar o classificador SVC
svc = SVC(random_state=42)

# Treinar o modelo
svc.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_svc = svc.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Acurácia do SVC: {accuracy_svc:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do SVC:")
print(classification_report(y_test, y_pred_svc))

Acurácia do SVC: 0.65

Relatório de Classificação do SVC:
              precision    recall  f1-score   support

           0       0.54      0.54      0.54       767
           1       0.51      0.57      0.54       766
           2       0.77      0.74      0.75       767
           3       0.81      0.74      0.77       767

    accuracy                           0.65      3067
   macro avg       0.66      0.65      0.65      3067
weighted avg       0.66      0.65      0.65      3067



In [14]:
# Inicializar o LDA
lda = LinearDiscriminantAnalysis()

# Treinar o modelo
lda.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_lda = lda.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_lda = accuracy_score(y_test, y_pred_lda)
print(f"Acurácia do LDA: {accuracy_lda:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do LDA:")
print(classification_report(y_test, y_pred_lda))

Acurácia do LDA: 0.70

Relatório de Classificação do LDA:
              precision    recall  f1-score   support

           0       0.57      0.60      0.59       767
           1       0.61      0.61      0.61       766
           2       0.80      0.78      0.79       767
           3       0.81      0.79      0.80       767

    accuracy                           0.70      3067
   macro avg       0.70      0.70      0.70      3067
weighted avg       0.70      0.70      0.70      3067



In [15]:
# Inicializar o RFC
rfc = RandomForestClassifier(random_state=42)

# Treinar o modelo
rfc.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_rfc = rfc.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_rfc = accuracy_score(y_test, y_pred_rfc)
print(f"Acurácia do RFC: {accuracy_rfc:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do RFC:")
print(classification_report(y_test, y_pred_rfc))

Acurácia do RFC: 0.64

Relatório de Classificação do RFC:
              precision    recall  f1-score   support

           0       0.52      0.53      0.53       767
           1       0.53      0.54      0.54       766
           2       0.76      0.75      0.76       767
           3       0.74      0.71      0.73       767

    accuracy                           0.64      3067
   macro avg       0.64      0.64      0.64      3067
weighted avg       0.64      0.64      0.64      3067



# Salvando

In [16]:
model_path = '/home/viniciuss/Documents/projetos/FER-2013_emotion_detection/lda_model.pkl'

# Salvar o modelo LDA treinado
with open(model_path, 'wb') as file:
    pickle.dump(lda, file)

print(f"Modelo LDA salvo em {model_path}")

Modelo LDA salvo em /home/viniciuss/Documents/projetos/FER-2013_emotion_detection/lda_model.pkl
